# Efficient Global Optimization employing Deep Gaussian Process Regression

See DGPR implementations in:

https://docs.gpytorch.ai/en/latest/examples/01_Exact_GPs/Simple_GP_Regression.html
https://docs.gpytorch.ai/en/stable/examples/06_PyTorch_NN_Integration_DKL/KISSGP_Deep_Kernel_Regression_CUDA.html

In [1]:
import numpy as np

from scipy.stats import qmc
from scipy.special import erf

import torch
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.utils.grid import ScaleToBounds
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.settings import use_toeplitz, fast_pred_var
from torch.optim import Adam

from BSA import bsa

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

## *GPR Model*

### Gaussian Process Regression

In [ ]:
class GPRegressionModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(MaternKernel(nu=1.5))
        self.scale_to_bounds = ScaleToBounds(-1., 1.)
        
        # Store train_y for later use
        self.train_outputs = train_y

    def forward(self, x):
        projected_x = self.scale_to_bounds(x)
        mean_x = self.mean_module(projected_x)
        covar_x = self.covar_module(projected_x)
        return MultivariateNormal(mean_x, covar_x)


### Train model

In [ ]:
def train_model(x, g, training_iterations):
    # Initialize the models and likelihood
    likelihood = GaussianLikelihood()
    model = GPRegressionModel(train_x=x, train_y=g, likelihood=likelihood)

    # if torch.cuda.is_available():
    #     model = model.cuda()
    #     likelihood = likelihood.cuda()

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = Adam([
        {'params': model.covar_module.parameters()},
        {'params': model.mean_module.parameters()},
        {'params': model.likelihood.parameters()},
    ], lr=0.01)

    # "Loss" for GPs - the marginal log likelihood
    mll = ExactMarginalLogLikelihood(likelihood, model)

    # Example training loop
    def train():
        loss_value = 1e8
        patience = round(training_iterations/50)
        wait = 0

        for _ in range(training_iterations):
            # Zero backprop gradients
            optimizer.zero_grad()

            # Get output from model
            output = model(x)

            # Calc loss and backprop derivatives
            loss = -mll(output, g)
            loss.backward()
            optimizer.step()

            # Callback
            if loss_value <= loss.item():
                wait += 1
            else:
                wait = 0

            if wait > patience:
                break

            loss_value = loss.item()

            # print(f"Iter {_+1} - Loss: {loss.item()}")
        print(f'Loss: {loss.item()}')
    train()

    # print('Test MAE: \
    # {}'.format(torch.mean(torch.abs(preds.mean - test_y))))
    return model, likelihood


## Initial sampling plan (Latin Hypercube sampling)

In [ ]:
def LHS_sample(num_points, DIM):
    # Number of variables and sampling points
    size = int(num_points)

    # Generate LHS samples
    sampler = qmc.LatinHypercube(d=DIM,
                                 optimization="random-cd")
    lhs_sample = sampler.random(n=size)

    return lhs_sample

## Expected Improvement

In [ ]:
def expected_improvement(x, model):
    """
    Function to calculate the Expected Improvement using Monte Carlo Integration.
    
    Parameters:
        x (array): Individual under evaluation.
        y (array): Valor mínimo obtido até o momento.
    
    Returns:
        array: The value of the Expected Improvement.
    """
    # Get the minimum value obtained so far
    ymin = torch.min(model.train_outputs)
    # ymin = torch.Tensor(ymin)

    # Calculate the prediction value and the variance (Ssqr)
    with torch.no_grad(), use_toeplitz(False), fast_pred_var():
        preds = model(x)
    f = preds.mean
    s = preds.variance

    # Check for any errors that are less than zero (due to numerical error)
    s[s < 0] = 0  # Set negative variances to zero

    # Calculate the RMSE (Root Mean Square Error)
    s = torch.sqrt(s)

    # Calculation of Expected Improvement
    term1 = (ymin - f) * (0.5 + 0.5 * erf((ymin - f) / (s * torch.sqrt( torch.from_numpy(np.array([2])) ))))
    term2 = (1 / torch.sqrt(2 * torch.from_numpy(np.array([np.pi])))) * s * torch.exp(-((ymin - f) ** 2) / (2 * s ** 2))
    
    return -(term1 + term2)

## Objective function

In [ ]:
def obj_fun(x):
    # Entra com x e constrói o modelo
    coordenadas(x)
    barras
    secoes
    materiais
    massa(x)
    
    # Análisa cada caso de carregamento no ANSYS
    P, h = 0, 1e5
    for carregamento in range(n_carregamentos):
        M, N, d = analisa_no_ANSYS(modelo)
        for elemento in range(n_elementos):
            if abs(d[elemento]) >= d_adm:
                P += h*(d[elemento] - d_adm)/d_adm
            
            if abs(N[elemento]) >= N_adm:
                P += h*(N[elemento] - N_adm)/N_adm
        
        P /= n_elementos
    P /= n_carregamentos
    
    # Penaliza (se precisar) o Momento, força axial e deslocamento de cada estaca,
    # para cada caso de carregamento e soma na penalização
    y = massa + P

    if isinstance(y, float):
        return torch.Tensor([y])
    return torch.Tensor(y)

## Define the problem

### Parameters of the optimization

In [ ]:
TRAINING_ITERATIONS = 2000  # epochs for training the GPR
N_INITIAL = 100  # initial number of points
N_INFILL = 400  # number of infill points
BSA_POPSIZE = 25
BSA_EPOCH = 500

### Set variables

In [ ]:
DIM = 18 # dimension of the problem
LOWER_BOUNDS = torch.zeros((DIM))
UPPER_BOUNDS = torch.ones((DIM))

### Generate initial sampling plan using LHS

In [ ]:
x = LHS_sample(N_INITIAL, DIM)
x = torch.from_numpy(x)
x = x.to(torch.float32)

In [ ]:
f = obj_fun(x)
f = f.view(N_INITIAL)

## Efficient Global Optimization

In [ ]:
model, likelihood = train_model(x, f, TRAINING_ITERATIONS)
model.eval()
likelihood.eval()

it = 0
print(f'\nIteration {it}. Best: {torch.min(f)} kg')

Loss: 1.009805323014733e+25

Iteration 0. Best: 4480.72314453125 kg


In [ ]:
while it < N_INFILL:
    it += 1
    
    # Search for the maximum expected improvement
    new_point = bsa(expected_improvement, low=LOWER_BOUNDS, up=UPPER_BOUNDS,
                    popsize=BSA_POPSIZE, epoch=BSA_EPOCH, data=model)
    x_new = torch.from_numpy(new_point.x)
    EI = new_point.y

    # Objective function at the new point
    f_new = obj_fun(x=x_new)
    
    print(f'Iteration {it} of {N_INFILL}')
    if f_new < torch.min(f): print(f'New best: {f_new:.2f} at position {it}')
    
    # Add new values to the initial sampling
    x = torch.cat((x, torch.from_numpy(np.array([np.asarray(x_new)]))), 0)
    x = x.to(torch.float32)
    f = torch.cat((f, f_new), 0)
    
    # Update model
    model, likelihood = train_model(x, f, TRAINING_ITERATIONS)
    model.eval()
    likelihood.eval()
    
    print(f'\nIteration {it}. Best: {torch.min(f)} kg')
    
    # if abs(EI) < TOL_MIN_EI:
    #     print('Optimization finished. Minimum tolerance achieved.')
    #     break

Iteration 1 of 200
Loss: 1.0076856768285132e+25

Iteration 1. Best: 4480.72314453125 kg
Iteration 2 of 200
Loss: 1.0040422142896547e+25

Iteration 2. Best: 4480.72314453125 kg
Iteration 3 of 200
Loss: 1.0017775305781555e+25

Iteration 3. Best: 4480.72314453125 kg
Iteration 4 of 200
Loss: 9.988835823094418e+24

Iteration 4. Best: 4480.72314453125 kg
Iteration 5 of 200
Loss: 1.0117107563853967e+25

Iteration 5. Best: 4480.72314453125 kg
Iteration 6 of 200
Loss: 1.0248187821398738e+25

Iteration 6. Best: 4480.72314453125 kg
Iteration 7 of 200
Loss: 1.0224989887804543e+25

Iteration 7. Best: 4480.72314453125 kg
Iteration 8 of 200
Loss: 1.0181254963448787e+25

Iteration 8. Best: 4480.72314453125 kg
Iteration 9 of 200
Loss: 1.0169700384129617e+25

Iteration 9. Best: 4480.72314453125 kg
Iteration 10 of 200
Loss: 1.0146694988426692e+25

Iteration 10. Best: 4480.72314453125 kg
Iteration 11 of 200
Loss: 1.011815672242316e+25

Iteration 11. Best: 4480.72314453125 kg
Iteration 12 of 200
Loss: 1.00

### Save results

In [ ]:
results = dict()
results["x"] = x
results["f"] = f
results["n_initial"] = N_INITIAL
results["n_infill"] = N_INFILL
results["OFEs"] = N_INITIAL + it

In [ ]:
print(f'f*: {torch.min(f)} kg; x*: {x[torch.argmin(f), :]}')

f*: 4480.72314453125 kg; x*: tensor([0.7097, 0.5206, 0.7261, 0.2552, 0.5334, 0.1589, 0.0682, 0.9707, 0.6417,
        0.7202, 0.3730, 0.8859, 0.2567, 0.7863, 0.8893, 0.6273, 0.8870, 0.6643,
        0.3576, 0.7533, 0.1121, 0.6268, 0.4831, 0.7478, 0.7098, 0.5823, 0.0339,
        0.6778, 0.8060, 0.4855, 0.3113, 0.6491, 0.1007, 0.7651, 0.4384, 0.0853,
        0.0815, 0.0304, 0.7789, 0.2740, 0.6065, 0.0547, 0.7040, 0.7034, 0.0755,
        0.7931, 0.5590, 0.3406, 0.6881, 0.9228, 0.9669, 0.7053])


# BSA

In [ ]:
BSA_POPSIZE = 12  # Ryzen 5 3600x (números múltiplos de 12)
BSA_EPOCH = 200  # depende do tempo computacional
results_BSA = bsa(obj_fun, low=LOWER_BOUNDS, up=UPPER_BOUNDS,
                    popsize=BSA_POPSIZE, epoch=BSA_EPOCH, data=[])
print(f'f*: {results_BSA.y} kg; x*: {results_BSA.x}')


KeyboardInterrupt



In [ ]:
plt.plot(np.arange(
         results_BSA.convergence.shape[0]),
         results_BSA.convergence,
         linewidth=2.5)

plt.xlabel('Iteration')
plt.ylabel('Cost')
# plt.ylim([ymin, ymax])

fig = plt.gcf()
fig.set_size_inches(16, 9)
plt.show()